# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import matplotlib.pyplot as plt
import torch
import glob, os
import numpy as np
import torch.nn.functional as F
from PIL import Image
from torch import nn
from torchvision import datasets, transforms, models
from collections import OrderedDict

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
means, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([transforms.RandomRotation(45),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means, std)])

#Test and validation use same transforms
test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(means, std)])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform = train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform = test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform = test_transforms)


# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [9]:
# Network Architecture #

#Introducing Pre-Trained VGG
model = models.vgg16(pretrained=True)

in_size = 25088
hidden_size = [5000, 1000]

dprob = 0.4

#Freezing Network parameters
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(in_size, hidden_size[0])),
                                        ('relu1', nn.ReLU()),
                                        ('dropout1', nn.Dropout(p=dprob)),
                                        ('fc2', nn.Linear(hidden_size[0], hidden_size[1])),
                                        ('relu2', nn.ReLU()),
                                        ('dropout2', nn.Dropout(p=dprob)),
                                        ('fc3', nn.Linear(hidden_size[1], 102))]))
#('output', nn.LogSoftmax(dim=1))
model.classifier = classifier


In [16]:
# Training the classifier #


epochs = 2
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=.001)


model = model.to('cuda')

steps = 0
for e in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        
        output = model.forward(images)
        
        loss = criterion(output, labels)
        loss.backward()
        
        running_loss += loss.item()
        
        optimizer.step()
        
        if steps % 100 == 0:
            #print('current accuracy for {} epoch: {}'.format(e + 1, accuracy))
            print('current total loss: {}'.format(running_loss/100))
            running_loss = 0
            
print('Training complete')

current total loss: -29.925986328125
current total loss: -38.3328857421875
current total loss: -48.2860546875
current total loss: -69.943828125
current total loss: -88.734423828125
current total loss: -95.298935546875
current total loss: -127.76189453125
current total loss: -144.546396484375
current total loss: -189.1309765625
current total loss: -213.2566796875
current total loss: -270.129609375
current total loss: -334.446171875
current total loss: -368.338046875
current total loss: -418.352734375
current total loss: -474.8430078125
current total loss: -564.5129296875


KeyboardInterrupt: 

In [15]:
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in validloader:
        images, labels = images.to('cuda'), labels.to('cuda')
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print (accuracy)

2.567237163814181


In [8]:
# Validation testing #
accuracy = 0
def validation(model, loader, criterion):
    total_loss = 0
    accuracy = 0
    model.eval()

    for images, labels in loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        output = model.forward(images)
        ps = torch.exp(output)

        loss = criterion(output, labels)
        total_loss += loss.item()
    
        is_right = (labels.data == ps.max(dim=1)[1])
        accuracy += is_right.type(torch.FloatTensor).mean()
        
        return total_loss, accuracy

print("The model's current accuracy is {}".format(accuracy))

The model's current accuracy is 0


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

model.class_to_idx = image_datasets['train'].class_to_idx

checkpoint = {'in_size': 25088,
             'hidden_size': [3136, 392],
             'epochs': 3,
             'optimizer_state': optimizer.state_dict(),
             'model_state': model.state_dict(),
             'classifier': classifier}

torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_model(filename):
    checkpoint = torch.load(filename)
    in_size = checkpoint['in_size']
    hidden_size = checkpoint['hidden_size']
    classifier = checkpoint['classifier']
    
    model = models.vgg16(pretrained=True)
    model.classifier = classifier
    model.load_state_dict(checkpoint['model_state'])
    
    return model

model = load_model('checkpoint.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [12]:
# I couldn't figure out how to use an image 
# from the given dataset, so this function 
# and the next is modified to work with one 
# of the images on the actual notebook.

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Scales
    width, height = image.size
    ratio = width / height
    
    if width >= height:
        height = 256
        width = ratio * height
    else:
        width = 256
        height = width / ratio
    
    size = width, height
    image.thumbnail(size)
    
    # Crops (left, top, right, bottom) 
    wdif = (width - 224) / 2
    hdif = (height - 224) / 2
    left, top, right, bot = wdif, hdif, width - wdif, height - hdif
    
    image = image.crop((left, top, right, bot))

    
    # Normalizes
    means, std = np.array([0.485, 0.456, 0.406]), np.array([0.229, 0.224, 0.225])
               
    image = np.array(image)
    print(image)
    image = image.transpose((2, 0, 1))
    print(image[0][34])
    
    for i in range(3):
        for n in range(224):
            image[i][n] = (image[i][n] - means[i]) / std[i]
    return image


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [13]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    #image = std * image + mean
    
    for i in range(3):
        for n in range(224):
            image[i][n] = (std[i] * image[i][n]) + means[i]
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

img = Image.open('assets/inference_example.png')
print(img)
img = process_image(img)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=558x838 at 0x7FB627F5AF60>
After scaling:  <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=256x384 at 0x7FB627F5AF60>
After cropping <PIL.Image.Image image mode=RGBA size=224x224 at 0x7FB627BB3E80>
[[[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [ 86  63  17 255]
  [113  79  33 255]
  [110  82  34 255]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [112  88  32 255]
  [147 110  51 255]
  [134 103  44 255]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [115  94  27 255]
  [143 113  41 255]
  [124  98  29 255]]

 ..., 
 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ..., 
  [255 255 255 25

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [7]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    output = model.forward(image_path)
    ps = F.softmax(output, dim=1)
    
    top5 = torch.topk(ps, topk, sorted = True)
    
    results = [cat_to_name[num] for num in top5]
    return top5, results



## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

sb.countplot()